In [14]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd

from stable_baselines3.ppo import PPO

PROJECT_ROOT_DIR = Path().absolute().parent.parent.parent
if str(PROJECT_ROOT_DIR.absolute()) not in sys.path:
    sys.path.append(str(PROJECT_ROOT_DIR.absolute()))

from utils_my.sb3.my_evaluate_policy import evaluate_policy_with_stat
from utils_my.sb3.vec_env_helper import get_vec_env

PROJECT_ROOT_DIR

PosixPath('/home/gxd/code/fly-craft-examples')

In [7]:
env_config = "D2D/env_config_for_sac_medium_b_05.json"
NUM_PROCESS = 16
SEED = 10


In [8]:
env_config_dict_in_training = {
    "num_process": NUM_PROCESS,
    "seed": SEED,
    "config_file": str(PROJECT_ROOT_DIR / "configs" / "env" / env_config),
    "custom_config": {"debug_mode": False, "flag_str": "Train"}
}

vec_env = get_vec_env(
    **env_config_dict_in_training
)

load config from: /home/gxd/code/fly-craft-examples/configs/env/D2D/env_config_for_sac_medium_b_05.json
16 Generator(PCG64) Generator(PCG64)
load config from: /home/gxd/code/fly-craft-examples/configs/env/D2D/env_config_for_sac_medium_b_05.json
13 Generator(PCG64) Generator(PCG64)
load config from: /home/gxd/code/fly-craft-examples/configs/env/D2D/env_config_for_sac_medium_b_05.json
15 Generator(PCG64) Generator(PCG64)
load config from: /home/gxd/code/fly-craft-examples/configs/env/D2D/env_config_for_sac_medium_b_05.json
11 Generator(PCG64) Generator(PCG64)
load config from: /home/gxd/code/fly-craft-examples/configs/env/D2D/env_config_for_sac_medium_b_05.json
25 Generator(PCG64) Generator(PCG64)
load config from: /home/gxd/code/fly-craft-examples/configs/env/D2D/env_config_for_sac_medium_b_05.json
17 Generator(PCG64) Generator(PCG64)
load config from: /home/gxd/code/fly-craft-examples/configs/env/D2D/env_config_for_sac_medium_b_05.json
10 Generator(PCG64) Generator(PCG64)
load config f

In [9]:
chkpt = PROJECT_ROOT_DIR / "checkpoints/disc/medium/ppo/epsilon_0_1_reg_0/128_128_2e8steps_seed_1/best_model.zip"

ppo_algo = PPO.load(chkpt)

In [13]:
_, _, _, res_dict_arr = evaluate_policy_with_stat(ppo_algo, vec_env, n_eval_episodes=16, deterministic=True)
res_dict_arr

/home/gxd/code/fly-craft-examples/utils_my/sb3/my_evaluate_policy.py:203: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


[{'last_info': {'step': 249,
   'is_success': True,
   'termination': 'reach_target_termination_single_step_based_on_angle_of_velocity_vector',
   'rewards': {'reach_target_termination_single_step_based_on_angle_of_velocity_vector': 0.0,
    '<rewards.dense_reward_based_on_angle_and_velocity.DenseRewardBasedOnAngleAndVelocity object at 0x7f9802682b10>': -0.10372511917798594},
   'action': {'p': 180.0, 'nz': 3.9828355, 'pla': 1.0, 'rud': 0.0},
   'desired_goal': [235.32397499056063,
    -11.620928068072573,
    56.379644184758604],
   'plane_state': {'lef': 6.638468532593751,
    'npos': 2985.620995563598,
    'epos': 179.69050096135803,
    'h': 865.6810687043514,
    'alpha': 12.587718452993377,
    'beta': 0.7152052316291165,
    'phi': 24.804428921271,
    'theta': 1.6254550821474425,
    'psi': 60.66563222156285,
    'p': -84.57881793858854,
    'q': 16.785339102017726,
    'r': -8.64550750971726,
    'v': 234.41451800780882,
    'vn': 131.0001144342243,
    've': 194.2118427968464

In [17]:
res_log = {
    "desired_goal": [],
    "is_success": [],
    "termination": [],
    "cumulative_reward": [],
    "episode_length": [],
}

for res_dict in res_dict_arr:
    res_log["desired_goal"].append(res_dict["last_info"]["desired_goal"])
    res_log["is_success"].append(res_dict["last_info"]["is_success"])
    res_log["termination"].append(res_dict["last_info"]["termination"])
    res_log["cumulative_reward"].append(res_dict["cumulative_reward"])
    res_log["episode_length"].append(res_dict["episode_length"])

res_df = pd.DataFrame(res_log)
res_df

,desired_goal,is_success,termination,cumulative_reward,episode_length
0,"[235.32397499056063, -11.620928068072573, 56.3...",True,reach_target_termination_single_step_based_on_...,-91.236093,249
1,"[239.1613900023811, 3.0840526378502346, -9.424...",True,reach_target_termination_single_step_based_on_...,-89.213379,249
2,"[215.4257149732124, 2.523752925327912, 42.1609...",True,reach_target_termination_single_step_based_on_...,-100.845373,283
3,"[196.19579095338642, 4.020216973637552, -5.770...",True,reach_target_termination_single_step_based_on_...,-98.990174,304
4,"[179.9345063499738, 3.6646196835797795, -11.11...",True,reach_target_termination_single_step_based_on_...,-118.014465,328
5,"[240.8621181088692, -1.960103249694864, 56.074...",False,negative_overload_and_big_phi_termination,-162.935946,373
6,"[215.60873517444037, -3.581885777907342, -59.9...",False,negative_overload_and_big_phi_termination,-174.277234,385
7,"[185.3274855060448, 5.495718236942608, -31.763...",False,timeout_termination,-118.809117,399
8,"[156.43556959236147, 26.201767495748022, 17.90...",False,timeout_termination,-169.659561,399
9,"[227.66861560008684, 28.75369735137376, 4.6011...",False,timeout_termination,-152.503191,399
